In [1]:
import warnings 
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import os
import random
import numpy as np
import yaml
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
import torch
import pytz
from datetime import datetime, timezone, timedelta
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder
from wandb.lightgbm import wandb_callback, log_summary

In [3]:
#wandb_callback 수정 
from typing import TYPE_CHECKING, Callable
import wandb
from wandb.sdk.lib import telemetry as wb_telemetry

MINIMIZE_METRICS = [
    "l1",
    "l2",
    "rmse",
    "mape",
    "huber",
    "fair",
    "poisson",
    "gamma",
    "binary_logloss",
]

MAXIMIZE_METRICS = ["map", "auc", "average_precision"]

def set_seeds(seed: int = 42):
    # 랜덤 시드를 설정하여 매 코드를 실행할 때마다 동일한 결과를 얻게 합니다.
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
        
def wandb_callback(log_params=True, define_metric=True) -> Callable:
    """Automatically integrates LightGBM with wandb.

    Arguments:
        log_params: (boolean) if True (default) logs params passed to lightgbm.train as W&B config
        define_metric: (boolean) if True (default) capture model performance at the best step, instead of the last step, of training in your `wandb.summary`

    Passing `wandb_callback` to LightGBM will:
      - log params passed to lightgbm.train as W&B config (default).
      - log evaluation metrics collected by LightGBM, such as rmse, accuracy etc to Weights & Biases
      - Capture the best metric in `wandb.summary` when `define_metric=True` (default).

    Use `log_summary` as an extension of this callback.

    Example:
        ```python
        params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            .
        }
        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=10,
                        valid_sets=lgb_eval,
                        valid_names=('validation'),
                        callbacks=[wandb_callback()])
        ```
    """
    def _define_metric(data: str, metric_name: str) -> None:
    
        """Capture model performance at the best step.
        instead of the last step, of training in your `wandb.summary`
        """
        if "loss" in str.lower(metric_name):
            wandb.define_metric(f"{data}_{metric_name}", summary="min")
        elif str.lower(metric_name) in MINIMIZE_METRICS:
            wandb.define_metric(f"{data}_{metric_name}", summary="min")
        elif str.lower(metric_name) in MAXIMIZE_METRICS:
            wandb.define_metric(f"{data}_{metric_name}", summary="max")
            
    log_params_list: "List[bool]" = [log_params]
    define_metric_list: "List[bool]" = [define_metric]

    def _init(env: "CallbackEnv") -> None:
        with wb_telemetry.context() as tel:
            tel.feature.lightgbm_wandb_callback = True

        wandb.config.update(env.params)
        log_params_list[0] = False

        if define_metric_list[0]:
            for i in range(len(env.evaluation_result_list)):
                data_type = env.evaluation_result_list[i][0]
                metric_name = env.evaluation_result_list[i][1]
                _define_metric(data_type, metric_name)

    def _callback(env: "CallbackEnv") -> None:
        if log_params_list[0]:
            _init(env)
        # eval_results: "Dict[str, Dict[str, List[Any]]]" = {}
        # recorder = lightgbm.record_evaluation(eval_results)
        # recorder(env)
        eval_results = {x[0]:{x[1:][0]:x[1:][1:]} for x in env.evaluation_result_list}

        for validation_key in eval_results.keys():
            for key in eval_results[validation_key].keys():
                 wandb.log(
                     {validation_key + "_" + key: eval_results[validation_key][key][0]},
                     commit=False,
                 )
        for item in eval_results:
            if len(item) == 4:
                wandb.log({f"{item[0]}_{item[1]}": item[2]}, commit=False)

        # Previous log statements use commit=False. This commits them.
        wandb.log({"iteration": env.iteration}, commit=True)

    return _callback

## Training

In [4]:
sweep_config_path = '/data/ephemeral/level2-dkt-recsys-06/code/boost/lgbmsweepconfigv2.yaml'

# 노트북의 이름 설정

os.environ['WANDB_NOTEBOOK_NAME'] = 'LGBM_Train.ipynb'
# YAML 파일 로드
with open(sweep_config_path, 'r') as file:
    sweep_config = yaml.safe_load(file)

# W&B 스위프트 설정
sweep_id = wandb.sweep(sweep=sweep_config, project="lightgbm-sweep")

# 시드 고정
set_seeds()


Create sweep with ID: j1rzdmbv
Sweep URL: https://wandb.ai/boostcamp6-recsys6/lightgbm-sweep/sweeps/j1rzdmbv


In [5]:
X = pd.read_csv('/data/ephemeral/level2-dkt-recsys-06/data/FE_v7.csv')
test =  pd.read_csv('/data/ephemeral/level2-dkt-recsys-06/data/FE_test_v7.csv')
X = X.sort_values(by=["userID", "Timestamp", "assessmentItemID"]).reset_index(drop=True)
test = test.sort_values(by=["userID", "Timestamp", "assessmentItemID"]).reset_index(drop=True)

# test = test[test["userID"] != test["userID"].shift(-1)]
# test = test.drop(["answerCode"], axis=1)

# 유저 아이디 갯수 구하기
unique_user_count = X['userID'].nunique()

# 유저아이디 갯수 출력
print(f'userIDnum {unique_user_count}')

# 유저아이디 마지막 행 구하기
last_rows = test.groupby('userID').tail(1)
print(last_rows)
# 마지막 행의 answerCode가 -1인지 확인
are_last_answers_minus_one = (last_rows['answerCode'] == -1).all()

print(are_last_answers_minus_one)

# test 모양
print(test.shape)



userIDnum 7442
        userID  assessmentItemID  testId  answerCode            Timestamp  \
1035         3          50133008   50133          -1  2020-10-26 13:13:57   
1706         4          70146008   70146          -1  2020-12-27 02:47:54   
3023        13          70111008   70111          -1  2020-12-27 04:35:09   
4283        17          90064006   90064          -1  2020-10-30 05:48:37   
4670        26          60135007   60135          -1  2020-10-23 11:44:18   
...        ...               ...     ...         ...                  ...   
260052    7395          40122005   40122          -1  2020-09-08 02:05:20   
260067    7404          30111005   30111          -1  2020-10-13 09:49:18   
260082    7416          50193004   50193          -1  2020-10-04 02:44:41   
260097    7417          50193004   50193          -1  2020-09-06 13:09:15   
260113    7439          40130005   40130          -1  2020-10-14 23:10:03   

        KnowledgeTag  Itemseq  SolvingTime  CumulativeTime  

In [6]:
test = test[test["answerCode"] == -1]
X = X[X['answerCode']!=-1]

In [7]:

# LabelEncoder 적용

label_encoders = {}
for column in [
    "categorize_ItemAnswerRate",
    "categorize_TagAnswerRate",
    "categorize_TestAnswerRate",
    "categorize_CumulativeUserItemAnswerRate",
    "categorize_CumulativeUserTagAverageAnswerRate",
    "categorize_CumulativeUserTagExponentialAverage",
    "DayOfWeek",
    "TimeOfDay",
    "categorize_TagAnswerRate",    
    "UserRecentTagAnswer",
    "PreviousItemAnswer",
    "categorize_TestAnswerRate",
]:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    # 테스트 데이터에 대해서는 transform만 적용
    test[column] = le.transform(test[column])
    label_encoders[column] = le


In [8]:
# print(X.shape)

# # 원-핫 인코딩 적용할 컬럼 선택
# columns_to_encode = [
#     "UserRecentTagAnswer",
#     "PreviousItemAnswer",
#     # 추가적으로 원-핫 인코딩을 적용할 다른 컬럼들을 여기에 추가
# ]
# for column in columns_to_encode:
#     if column in X.columns:
#         X = pd.get_dummies(X, columns=[column])
#     if column in test.columns:
#         test = pd.get_dummies(test, columns=[column])

#         # 다른 필드들에 대해서도 동일하게 적용
# print(X.shape)

# 라벨로 이동 


In [9]:
feat = X.columns.tolist()

exclude_columns = [
    "Timestamp",
    "answerCode",
    "DayOfWeek",
    'WeekOfYear',
    'UserAvgSolvingTime',
    'PastItemCount',
    "user_tag_total_answer",
    "categorize_CumulativeUserTagExponentialAverage",
    'categorize_CumulativeUserTagAverageAnswerRate',
    "categorize_TestAnswerRate",
    "categorize_TagAnswerRate"
]

filtered_feat = [column for column in feat if column not in exclude_columns]

print(X[feat].shape)
print(X[filtered_feat].shape)
print(X.columns.tolist())

(2525956, 54)
(2525956, 43)
['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'KnowledgeTag', 'Itemseq', 'SolvingTime', 'CumulativeTime', 'Month', 'DayOfWeek', 'TimeOfDay', 'WeekOfYear', 'UserAvgSolvingTime', 'CumulativeItemCount', 'Item_last7days', 'Item_last30days', 'PastItemCount', 'CumulativeUserItemAnswerRate', 'ItemAnswerRate', 'AverageItemSolvingTime_Correct', 'AverageItemSolvingTime_Incorrect', 'AverageItemSolvingTime', 'Difference_SolvingTime_AvgItemSolvingTime', 'UserTagAvgSolvingTime', 'TagAnswerRate', 'CumulativeUserTagAverageAnswerRate', 'CumulativeUserTagExponentialAverage', 'UserCumulativeTagCount', 'UserRecentTagAnswer', 'PreviousItemAnswer', 'TestAnswerRate', 'categorize_solvingTime', 'categorize_ItemAnswerRate', 'categorize_TagAnswerRate', 'categorize_TestAnswerRate', 'categorize_CumulativeUserItemAnswerRate', 'categorize_CumulativeUserTagAverageAnswerRate', 'categorize_CumulativeUserTagExponentialAverage', 'user_test_correct_answer', 'user_test_tota

In [10]:
default_config = {
    "num_leaves": 10,  # 최소값 10
    "learning_rate": 0.0001,  # 최소값 0.0001
    "max_depth": -1,  # -1 (깊이 제한 없음)
    "min_data_in_leaf": 20,  # 최소값 20
    "feature_fraction": 0.6,  # 최소값 0.6
    "bagging_fraction": 0.6,  # 최소값 0.6
    "bagging_freq": 0,  # 최소값 0
    "lambda_l1": 0.0,  # 최소값 0.0
    "lambda_l2": 0.0,  # 최소값 0.0
    "cat_smooth": 10,  # 최소값 10
}



In [11]:
def train():
    auc = 0
    acc = 0
    test_preds = np.zeros(len(test))

    wandb.init(project=f"lightgbm-sweep", config=default_config)

    ratio = wandb.config.ratio

    sampled_indices = X.groupby("userID").sample(frac=ratio).index

    # userID별 마지막 인덱스 찾기
    # last_indices = X.groupby("userID").tail(1).index

    # 학습 데이터셋 생성
    X_train = X.drop(sampled_indices)
    y_train = X_train["answerCode"]

    # 검증 데이터셋 생성
    X_valid = X.loc[sampled_indices]
    y_valid = X_valid["answerCode"]

    lgb_train = lgb.Dataset(X_train[filtered_feat], y_train)
    lgb_valid = lgb.Dataset(X_valid[filtered_feat], y_valid)

    # 완드비 실험 이름
    korea = pytz.timezone("Asia/Seoul")
    current_time = datetime.now(korea).strftime("%m-%d %H:%M")
    wandb.run.name = f"wooksbaby-{current_time},ratio-{ratio}"
    current_params = {
        "objective": "binary",
        "metric": ["auc"],
        "device": "cpu",
        "num_leaves": wandb.config.num_leaves,
        "learning_rate": wandb.config.learning_rate,
        "max_depth": wandb.config.max_depth,
        "min_data_in_leaf": wandb.config.min_data_in_leaf,
        "feature_fraction": wandb.config.feature_fraction,
        "bagging_fraction": wandb.config.bagging_fraction,
        "bagging_freq": wandb.config.bagging_freq,
        "lambda_l1": wandb.config.lambda_l1,
        "lambda_l2": wandb.config.lambda_l2,
        "cat_smooth": wandb.config.cat_smooth,
    }
    model = lgb.train(
        current_params,
        lgb_train,
        valid_sets=[lgb_train, lgb_valid],
        num_boost_round=1500,
        callbacks=[
            wandb_callback(log_params=True, define_metric=True),
            lgb.early_stopping(20),
        ],
        categorical_feature=[
            "userID",
            "assessmentItemID",
            "testId",
            "KnowledgeTag",
            "Month",
        ],
    )
    preds = model.predict(X_valid[filtered_feat])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)
    test_preds += model.predict(test[filtered_feat])
    print(f"VALID AUC : {auc} ACC : {acc}\n")

    # output파일 생성
    output_dir = "output/"
    write_path = os.path.join(
        output_dir,
        f"auc:{auc} acc:{acc}" + "sweep" + " lgbm.csv",
    )
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    with open(write_path, "w", encoding="utf8") as w:
        print("writing prediction : {}".format(write_path))
        w.write("id,prediction\n")
        for id, p in enumerate(test_preds):
            w.write("{},{}\n".format(id, p))

    # feature_importances = model.feature_importance()
    # feature_names = model.feature_name()
    # importance_df = pd.DataFrame(
    #     {"Feature": feature_names, "Importance": feature_importances}
    # ).sort_values(by="Importance", ascending=False)
    # print(importance_df)

    wandb.log({"auc": auc, "accuracy": acc})
    wandb.finish()

In [12]:

wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 4nagb56u with config:
wandb: 	bagging_fraction: 0.7100504474382494
wandb: 	bagging_freq: 5
wandb: 	feature_fraction: 0.6880100231147456
wandb: 	lambda_l1: 2.9408358264506456
wandb: 	lambda_l2: 9.61425685345089
wandb: 	learning_rate: 0.4472517690379719
wandb: 	max_depth: 48
wandb: 	min_data_in_leaf: 82
wandb: 	num_leaves: 36
wandb: 	ratio: 0.050189620831581586
wandb: Currently logged in as: wooksbaby (boostcamp6-recsys6). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] Number of positive: 1570686, number of negative: 828425
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057523 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22936
[LightGBM] [Info] Number of data points in the train set: 2399111, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654695 -> initscore=0.639741
[LightGBM] [Info] Start training from score 0.639741


wandb: WARNING Config item 'num_leaves' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_depth' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_data_in_leaf' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'feature_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_freq' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lambda_l1' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lambda_l2' was locked by 'sweep' (ignored update).


[1]	training's auc: 0.823968	valid_1's auc: 0.823895
Training until validation scores don't improve for 20 rounds
[2]	training's auc: 0.834339	valid_1's auc: 0.834243
[3]	training's auc: 0.839193	valid_1's auc: 0.839058
[4]	training's auc: 0.842217	valid_1's auc: 0.841779
[5]	training's auc: 0.844997	valid_1's auc: 0.844208
[6]	training's auc: 0.847135	valid_1's auc: 0.845797
[7]	training's auc: 0.848908	valid_1's auc: 0.847225
[8]	training's auc: 0.850447	valid_1's auc: 0.848354
[9]	training's auc: 0.851801	valid_1's auc: 0.849308
[10]	training's auc: 0.85317	valid_1's auc: 0.850215
[11]	training's auc: 0.85435	valid_1's auc: 0.850992
[12]	training's auc: 0.855328	valid_1's auc: 0.851585
[13]	training's auc: 0.856237	valid_1's auc: 0.851986
[14]	training's auc: 0.856816	valid_1's auc: 0.852298
[15]	training's auc: 0.85761	valid_1's auc: 0.85268
[16]	training's auc: 0.858432	valid_1's auc: 0.853173
[17]	training's auc: 0.859195	valid_1's auc: 0.853542
[18]	training's auc: 0.85975	valid

accuracy,▁
auc,▁
iteration,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
training_auc,▁▃▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████
valid_1_auc,▁▄▆▆▇▇▇█████████████████████████████████
accuracy,0.80202
auc,0.8567
iteration,108


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: lrpej6rd with config:
wandb: 	bagging_fraction: 0.5712523457191416
wandb: 	bagging_freq: 21
wandb: 	feature_fraction: 0.7694977001003088
wandb: 	lambda_l1: 2.4574444727082696
wandb: 	lambda_l2: 0.4906413940801369
wandb: 	learning_rate: 0.3692315632600192
wandb: 	max_depth: 44
wandb: 	min_data_in_leaf: 93
wandb: 	num_leaves: 45
wandb: 	ratio: 0.029726521648650563
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] Number of positive: 1604161, number of negative: 846605
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058615 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22923
[LightGBM] [Info] Number of data points in the train set: 2450766, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654555 -> initscore=0.639122
[LightGBM] [Info] Start training from score 0.639122


wandb: WARNING Config item 'num_leaves' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_depth' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_data_in_leaf' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'feature_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_fraction' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'bagging_freq' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lambda_l1' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lambda_l2' was locked by 'sweep' (ignored update).


[1]	training's auc: 0.82657	valid_1's auc: 0.823056
Training until validation scores don't improve for 20 rounds
[2]	training's auc: 0.83618	valid_1's auc: 0.832964
[3]	training's auc: 0.84038	valid_1's auc: 0.836633
[4]	training's auc: 0.843045	valid_1's auc: 0.839307
[5]	training's auc: 0.84562	valid_1's auc: 0.841678
[6]	training's auc: 0.847873	valid_1's auc: 0.843442
[7]	training's auc: 0.849672	valid_1's auc: 0.844765
[8]	training's auc: 0.851073	valid_1's auc: 0.845999
[9]	training's auc: 0.852519	valid_1's auc: 0.84712
[10]	training's auc: 0.85395	valid_1's auc: 0.848153
[11]	training's auc: 0.855266	valid_1's auc: 0.849002
[12]	training's auc: 0.856536	valid_1's auc: 0.849858
[13]	training's auc: 0.857483	valid_1's auc: 0.850311
[14]	training's auc: 0.858264	valid_1's auc: 0.850809
[15]	training's auc: 0.858992	valid_1's auc: 0.851105
[16]	training's auc: 0.85994	valid_1's auc: 0.85163
[17]	training's auc: 0.860446	valid_1's auc: 0.851764
[18]	training's auc: 0.860959	valid_1'